#### **Image Dataset Ingestion**

This notebook handles the ingestion of the image dataset for classification purposes. The dataset includes  images of various chicken diseases, stored in different directories .


#### **Changing Directory in Python**

The following Python code demonstrates changing the working directory using the `os` module in Python.


In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\SURESH BEEKHANI\\Desktop\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\SURESH BEEKHANI\\Desktop\\Chicken-Disease-Classification'

#### **Data Ingestion Configuration**

This section defines a configuration class for data ingestion using Python's `dataclasses` and `pathlib` modules.


In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path          # The root directory for data storage
    source_URL: str         # URL to the data source
    local_data_file: Path   # Path to the local data file
    unzip_dir: Path         # Directory where data will be extracted


#### **Configuration Manager**

The `ConfigurationManager` class is responsible for managing configuration settings for the project. It reads configuration files, sets up necessary directories, and provides configurations for different stages of the data pipeline.


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    # The ConfigurationManager class handles configuration-related tasks, such as reading configuration files
    # and setting up directories for data ingestion and other processes in the project.

    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        """
        Initializes ConfigurationManager with paths to configuration and parameters files.
        
        :param config_filepath: Path to the main configuration YAML file.
        :param params_filepath: Path to the parameters YAML file.
        """
        
        # Read the YAML configuration file and store its content in self.config
        self.config = read_yaml(config_filepath)
        
        # Read the YAML parameters file and store its content in self.params
        self.params = read_yaml(params_filepath)

        # Create the main artifacts directory specified in the config, if it doesn't exist
        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Retrieves and prepares the data ingestion configuration.

        :return: An instance of DataIngestionConfig containing the data ingestion settings.
        """
        
        # Access the data ingestion part of the configuration
        config = self.config.data_ingestion

        # Ensure that the root directory for data ingestion exists or create it
        create_directories([config.root_dir])

        # Create a DataIngestionConfig instance with necessary parameters for data ingestion
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,            # Root directory for data storage
            source_URL=config.source_URL,        # URL for downloading data
            local_data_file=config.local_data_file,  # Path to the local data file
            unzip_dir=config.unzip_dir           # Directory to store unzipped data files
        )

        # Return the configured DataIngestionConfig instance
        return data_ingestion_config


#### **Data Ingestion**

The `DataIngestion` class handles the process of downloading and extracting a ZIP file dataset from a specified URL. It leverages several modules to manage file paths, perform download operations, handle ZIP files, and log events.


In [7]:
import os  # Import the 'os' module for interacting with the operating system, such as file and directory operations.

import urllib.request as request  # Import 'urllib.request' as 'request' for handling URL operations, 
# such as opening and reading URLs, which is useful for downloading files from the internet.

import zipfile  # Import the 'zipfile' module to work with ZIP archives, enabling the ability to read and extract ZIP files.

from cnnClassifier import logger  # Import the 'logger' from the 'cnnClassifier' module for logging purposes,
# allowing you to record messages about the execution of the program (e.g., errors, info messages).

from cnnClassifier.utils.common import get_size  # Import the 'get_size' function from the 'common' module in 'utils' of 'cnnClassifier'.
# This function likely retrieves the size of a file or directory, which can be useful for managing resources or logging.
from pathlib import Path

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

#### **Data Ingestion Process Execution**

This block of code initializes and runs the data ingestion process using the `ConfigurationManager` and `DataIngestion` classes. Error handling is included to catch and raise any exceptions that may occur during execution.


In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-26 23:53:23,238: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-26 23:53:23,245: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-26 23:53:23,248: INFO: common: created directory at: artifacts]
[2024-10-26 23:53:23,251: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-26 23:53:55,912: INFO: 2754697353: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BD0F:1A6FF7:A7CB9:1A1502:671DDE8B
Accept-Ranges: bytes
Date: Sun, 27 Oct 2024 06:53:25 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990031-FJR
X-Cache: 